In [ ]:
import os
import time
from PIL import Image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.manifold import Isomap
from sklearn.decomposition import PCA
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import umap.umap_ as umap

feature_limit = 750

brain_list = []
typelist = ['glioma', 'meningioma', 'pituitary', 'notumor']
for x in typelist:
    for image in os.listdir(f"archive/brain-tumor-mri-dataset/{x}")[:feature_limit]:
        image_path = os.path.join(f"archive/brain-tumor-mri-dataset/{x}", image)
        img_arr = np.array(Image.open(image_path).resize((64,64)))
        if len(img_arr.shape) == 3:
            img_arr = img_arr[:, :, 0]
    
        #print(img_arr.shape)
        #plt.imshow(img_arr)
        #plt.show()
        brain_list.append(img_arr.flatten())

x = np.array(brain_list) / 255



In [6]:
#create label arrays:
gli = np.ones((feature_limit, 1))
men = np.ones((feature_limit, 1))
pit = np.ones((feature_limit, 1))
notu = np.zeros((feature_limit, 1))

# two or four categories (Cancer or not) Using MLP Classifier
#y = np.vstack((gli, men, pit, notu))
y = np.vstack((gli, men*2, pit*3, notu))[:, 0]

xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size = 0.2)
#print(x)


In [7]:
perceptron = MLPClassifier(hidden_layer_sizes=(64, 30), learning_rate='invscaling')
perceptron.fit(xtrain, ytrain.flatten())
perceptron_pred = perceptron.predict(xtest)
acc_perceptron = accuracy_score(ytest.flatten(), perceptron_pred)
f1_perceptron = f1_score(ytest, perceptron_pred, average='weighted')
print('perceptron acc/f1')
print(acc_perceptron)
print(f1_perceptron)


adaboost = AdaBoostClassifier(n_estimators=100, random_state=12)
adaboost.fit(xtrain, ytrain.flatten())
ada_pred = adaboost.predict(xtest)
acc_adaboost = accuracy_score(ytest.flatten(), ada_pred)
f1_adaboost = f1_score(ytest, ada_pred, average='weighted')
print('adaboost acc/f1')
print(acc_adaboost)
print(f1_adaboost)

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(xtrain, ytrain)
knn_pred = knn.predict(xtest)
acc_knn = accuracy_score(ytest, knn_pred)
f1_knn = f1_score(ytest, knn_pred, average='weighted')
print('KNN acc/f1')
print(acc_knn)
print(f1_knn)

perceptron acc/f1
0.8533333333333334
0.8529095801111932
adaboost acc/f1
0.7366666666666667
0.7393689317070788
KNN acc/f1
0.8116666666666666
0.8028490952984769


In [9]:
modelresults = {'Perceptron':  {'accuracy':acc_perceptron, 'f1_score':f1_perceptron}, 
                'Adaboost':    {'accuracy':acc_adaboost,   'f1_score':f1_adaboost},  
                'KNN':         {'accuracy':acc_knn,        'f1_score':f1_knn}}

display(pd.DataFrame(modelresults).T.sort_values(by='accuracy', ascending=False))

,accuracy,f1_score
Perceptron,0.853333,0.852910
KNN,0.811667,0.802849
Adaboost,0.736667,0.739369


In [ ]:
# 4 categories (glioma, meningioma, pituitary, notumor)
y = np.vstack((gli, men*2, pit*3, notu))[:, 0]
#y = y.reshape((1200, 1))


In [ ]:
#x = x / 255
x = x.reshape(-1, 64, 64, 1)
#print(x.shape)

In [ ]:
# Split data in training / testing sets

xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size = 0.2)

In [ ]:
# 4 classes
model2 = Sequential([
    Input(shape=(64, 64, 1)),
    
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(128, (3, 3), activation='relu'), 
    MaxPooling2D(pool_size=(2, 2)),
    
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.2),
    Dense(4, activation='softmax')  
])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
model2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Stop if val_loss doesn't improve for 3 epochs
early_stop = EarlyStopping(monitor='val_loss', patience=4, restore_best_weights=True)

history = model2.fit(xtrain, ytrain, epochs=20, batch_size=32, validation_data=(xtest, ytest), callbacks=[early_stop])

In [ ]:
loss, accuracy = model2.evaluate(xtest, ytest)
print(f"Test accuracy: {accuracy:.4f}")

y_pred = model2.predict(xtest)
y_pred = np.argmax(y_pred, axis=1)
ytest = ytest.astype('int')
cnn_acc = accuracy_score(ytest, y_pred)
f1_cnn = f1_score(ytest, y_pred, average='weighted')

modelresults['CNN'] = {'accuracy':cnn_acc, 'f1_score':f1_cnn}

display(pd.DataFrame(modelresults).T.sort_values(by='accuracy', ascending=False))

In [ ]:
num_images = 16
misclassified = np.where(y_pred != ytest)
correct = np.where(y_pred == ytest)

plt.figure(figsize=(12, 12))
for i in range(num_images):
    plt.subplot(4, 4, i + 1)
    plt.suptitle(f'Correct Classification samples: {cnn_acc * 100:.2f}% of test images', fontsize=16, fontweight='bold')
    img = xtest[i].reshape(64, 64)  
    plt.imshow(img, cmap='gray')
    plt.title(f"Pred: {y_pred[correct[0][i]]}\nTrue: {ytest[correct[0][i]]}", 
              color='green' if y_pred[correct[0][i]] == ytest[correct[0][i]] else 'red')
    plt.axis('off')
plt.tight_layout()
plt.show()

print()
plt.figure(figsize=(12, 12))
for i in range(num_images):
    plt.subplot(4, 4, i + 1)
    plt.suptitle(f'Incorrect Classification samples: {(1 - cnn_acc)*100:.2f}% of test images', fontsize=16, fontweight='bold')
    img = xtest[misclassified[0][i]].reshape(64, 64)  
    plt.imshow(img, cmap='gray')
    plt.title(f"Pred: {y_pred[misclassified[0][i]]}\nTrue: {ytest[misclassified[0][i]]}", 
              color='green' if y_pred[misclassified[0][i]] == ytest[misclassified[0][i]] else 'red')
    plt.axis('off')
plt.tight_layout()
plt.show()


In [ ]:
reducer = umap.UMAP(n_neighbors=11, n_components=4)
x_umap = reducer.fit_transform(x.reshape((-1, 64*64)))


plt.scatter(x_umap[:, 0], x_umap[:, 1], c=y)
plt.plot()

# 3d scatterplot
fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111, projection='3d')
scatter = ax.scatter(x_umap[:, 0], x_umap[:, 1], x_umap[:, 2], c=y)
ax.legend(*scatter.legend_elements(), title='Legend')
plt.show()

In [ ]:
isomap = Isomap(n_neighbors=14, n_components=3)
x_iso = isomap.fit_transform(x.reshape((-1, 64*64)))
#print(x_iso.shape)

# 2d scatterplot
plt.scatter(x_iso[:, 0], x_iso[:, 1], c=y)
plt.plot()

# 3d scatterplot
fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111, projection='3d')
scatter = ax.scatter(x_iso[:, 0], x_iso[:, 1], x_iso[:, 2], c=y)
ax.legend(*scatter.legend_elements(), title='Legend')
plt.show()

In [ ]:
pca = PCA(n_components=3)
x_pca = pca.fit_transform(x.reshape((-1, 64*64)))

plt.scatter(x_pca[:, 0], x_pca[:, 1], c=y)
plt.plot()

# 3d scatterplot
fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111, projection='3d')
scatter = ax.scatter(x_pca[:, 0], x_pca[:, 1], x_pca[:, 2], c=y)
ax.legend(*scatter.legend_elements(), title='Legend')
plt.show()

In [ ]:
brain_ex = Image.open("C:/Users/ryana/OneDrive/Desktop/GaTech OMSA/Spring '25/ISYE6740 - CDA/Project/brain_samp.jpg").convert("L")

background = Image.new("L", (427, 427))
#plt.imshow(background)
background.paste(brain_ex, (((427-341) // 2, 0)))

brain_ex = np.array(background.resize((64, 64))).reshape((64, 64, 1)) / 255
print(brain_ex.shape)
plt.imshow(brain_ex, cmap='gray')
plt.plot()



In [ ]:
brain_ex = np.expand_dims(brain_ex, axis=0)
brainex_pred = model2.predict(brain_ex)
brainex_pred = np.argmax(brainex_pred, axis=1)
print(brainex_pred)